In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [50]:
driver = webdriver.Chrome("chromedriver")

In [70]:
# define an isin list
isin = ["DE000PS7WT17", "IE00BKM4GZ66"]
# isin = ["IE00BKM4GZ66"]

In [69]:
def get_etp_data(isin_list, start_date = "01.01.2000"):
    """Scrapes historical prices for an ETP form the boerse-frankfurt website."""
    
    # open xetra page
    driver.get("https://www.boerse-frankfurt.de/")
    
    # accept cookies
    cookies = driver.find_element_by_id("cookie-hint-btn-accept")
    cookies.click()
    
    for isin in isin_list:
        df = pd.DataFrame()

        # open webpage for etp
        driver.get(f"https://www.boerse-frankfurt.de/etp/{isin}")

        # open Kurshistorie
        history = driver.find_element_by_xpath("//button[text()='Kurshistorie']")
        history.click()

        # set start date
        start = driver.find_elements_by_class_name("text-center")[1]
        start.click()
        start.clear()
        start.send_keys(start_date)

        # reload from start_date
        submit = driver.find_element_by_xpath("//button[@class = 'btn btn-sm form-button']")
        submit.click()

        # get 100 days per page
        more_results = driver.find_element_by_xpath("//button[text()='100']")
        more_results.click()
        
        # append to df
        # df = df.append(pd.read_html(driver.page_source)[0])
        
        # select next page
        next_page = driver.find_element_by_xpath("//button[@title= 'Zeige Seite 2']")
        
        # scrape table for each page
        while next_page.is_enabled():
            next_page.click()
            df = df.append(pd.read_html(driver.page_source)[0])
        
        # scrape last page
        df = df.append(pd.read_html(driver.page_source)[0])
        
        # save as csv
        df.to_csv(f"{isin}.csv", index = False)
        
    return df


In [71]:
df = get_etp_data(isin)

In [72]:
df

,Datum,Eröffnung,Schluss,Tageshoch,Tagestief,Umsatz in €,Umsatz in Stück
0,15.05.20,22596,22255,22596,22216,3.085.598,137.594
1,14.05.20,22445,22358,22494,2208,3.048.945,136.902
2,13.05.20,22699,22623,22809,22574,5.190.605,228.504
3,12.05.20,22597,22856,22861,22593,3.116.641,137.127
4,11.05.20,22852,22723,22898,22565,4.078.103,179.402
...,...,...,...,...,...,...,...
97,10.06.14,1894,1889,1894,1889,13.258,700
98,09.06.14,1861,1871,1871,1861,4.987,268
99,06.06.14,1843,1858,1862,1843,29.379,1.588
0,05.06.14,1841,1841,1841,1841,NaN,NaN
